In [75]:
import pandas as pd
import numpy as np

import unidecode
import re
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LinearRegression
import sklearn.metrics as sm
from sklearn.metrics import r2_score

In [76]:
fb_df = pd.read_csv('data/Premier-League-2015-2019.csv')


In [77]:
fb_df.sample(10)

,Date,HomeTeam,AwayTeam,outcome_by_HOME,AVERAGE_ODD_WIN,AVERAGE_ODD_DRAW,AVERAGE_ODD_OPPONENT_WIN,RANKINGHOME,RANKING AWAY,LAST_GAME_RHOMETEAM,LAST_GAME_RAWAYTEAM
425,17/09/2016,West Brom,West Ham,W,8.691,3.145,2.784,14,7,-1,1
663,01/04/2017,Burnley,Tottenham,L,9.946,4.055,2.116,20,3,1,-1
344,24/04/2016,Sunderland,Arsenal,D,9.192,3.954,2.096,16,3,1,0
7,09/08/2015,Newcastle,Southampton,D,7.630,3.198,2.689,15,7,0,0
1325,26/12/2018,Liverpool,Newcastle,W,7.996,10.482,14.688,4,10,1,0
416,10/09/2016,Stoke,Tottenham,L,10.332,3.737,2.141,9,3,0,-1
440,30/09/2016,Everton,Crystal Palace,D,7.875,3.610,4.688,11,15,0,-1
1333,29/12/2018,Liverpool,Arsenal,W,6.439,7.740,5.272,4,6,1,0
358,07/05/2016,Crystal Palace,Stoke,W,7.384,3.261,3.649,10,9,1,1
927,13/12/2017,Swansea,Man City,L,15.419,7.303,2.576,15,3,1,1


In [78]:
fb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1520 entries, 0 to 1519
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Date                      1520 non-null   object 
 1   HomeTeam                  1520 non-null   object 
 2   AwayTeam                  1520 non-null   object 
 3   outcome_by_HOME           1520 non-null   object 
 4   AVERAGE_ODD_WIN           1520 non-null   float64
 5   AVERAGE_ODD_DRAW          1520 non-null   float64
 6   AVERAGE_ODD_OPPONENT_WIN  1520 non-null   float64
 7   RANKINGHOME               1520 non-null   int64  
 8   RANKING AWAY              1520 non-null   int64  
 9   LAST_GAME_RHOMETEAM       1520 non-null   int64  
 10  LAST_GAME_RAWAYTEAM       1520 non-null   int64  
dtypes: float64(3), int64(4), object(4)
memory usage: 130.8+ KB


Check for duplicates

In [79]:
fb_df.duplicated().sum()

0

Checking for missing values

In [80]:
fb_df.isnull().sum()

Date                        0
HomeTeam                    0
AwayTeam                    0
outcome_by_HOME             0
AVERAGE_ODD_WIN             0
AVERAGE_ODD_DRAW            0
AVERAGE_ODD_OPPONENT_WIN    0
RANKINGHOME                 0
RANKING AWAY                0
LAST_GAME_RHOMETEAM         0
LAST_GAME_RAWAYTEAM         0
dtype: int64

 normalize and standardize column names

In [81]:
fb_df.rename(columns={
    'Date': 'date',
    'HomeTeam': 'home_team',
    'AwayTeam': 'away_team',
    'outcome_by_HOME': 'home_outcome',
    'AVERAGE_ODD_WIN': 'avg_odd_home_win',
    'AVERAGE_ODD_DRAW': 'avg_odd_draw',
    'AVERAGE_ODD_OPPONENT_WIN': 'avg_odd_away_win',
    'RANKINGHOME': 'home_ranking',
    'RANKING AWAY': 'away_ranking',
    'LAST_GAME_RHOMETEAM': 'last_home_result',
    'LAST_GAME_RAWAYTEAM': 'last_away_result'
}, inplace=True)

In [82]:
fb_df.head()

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
0,08/08/2015,Bournemouth,Aston Villa,L,7.024,3.303,3.748,20,17,0,0
1,08/08/2015,Chelsea,Swansea,D,6.697,4.338,8.967,1,8,0,0
2,08/08/2015,Everton,Watford,D,6.842,3.514,4.852,11,20,0,0
3,08/08/2015,Leicester,Sunderland,W,7.000,3.207,3.742,14,16,0,0
4,08/08/2015,Man United,Tottenham,W,6.477,3.651,5.318,4,5,0,0


Label encoding

label encode home and away team and make sure a team in Home will have the same ID in Away

In [83]:
all_teams = pd.concat([fb_df['home_team'], fb_df['away_team']]).unique()

team_encoder = LabelEncoder()
team_encoder.fit(all_teams)

fb_df['home_team'] = team_encoder.transform(fb_df['home_team'])
fb_df['away_team'] = team_encoder.transform(fb_df['away_team'])

In [84]:
fb_df.sample(10)

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
1078,07/04/2018,14,15,L,6.287,3.464,4.018,3,6,1,1
1121,06/05/2018,0,4,W,6.425,3.890,5.954,5,16,-1,0
1245,03/11/2018,26,4,W,6.448,7.105,4.400,13,7,0,-1
304,02/04/2016,1,6,L,10.431,4.302,2.098,17,1,1,-1
1435,09/03/2019,19,23,W,7.321,7.144,2.136,17,3,-1,0
223,23/01/2016,18,13,L,8.751,3.526,2.211,20,6,1,1
510,03/12/2016,7,19,W,8.051,3.353,2.421,15,6,-1,-1
632,25/02/2017,8,21,W,6.972,4.514,7.436,11,17,0,-1
365,08/05/2016,23,19,L,7.349,3.408,3.681,5,7,-1,-1
1205,29/09/2018,17,12,L,6.913,7.212,2.822,10,9,0,1


Label encoding for home_outcome

In [85]:
#label encoding for home_outcome
df_label = fb_df
df_label['home_outcome'] = df_label['home_outcome'].map({'L': 0, 'D': 1, 'W': 2})

In [86]:
df_label.sample(10)

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
1418,27/02/2019,19,9,2,6.095,6.811,3.756,17,20,-1,-1
249,07/02/2016,6,15,1,7.090,3.130,3.646,1,4,0,0
986,13/01/2018,25,3,2,6.049,2.953,3.965,10,20,-1,0
666,01/04/2017,12,20,2,7.138,3.291,3.737,1,9,0,1
672,02/04/2017,22,16,1,7.088,3.259,4.017,12,20,1,-1
1154,18/08/2018,23,9,2,7.417,9.088,9.057,3,20,1,-1
1193,22/09/2018,7,17,1,6.700,7.241,3.363,11,10,1,-1
915,09/12/2017,23,20,2,5.998,5.470,9.871,2,13,0,1
456,15/10/2016,25,23,1,10.082,3.917,2.131,14,3,-1,0
725,06/05/2017,2,20,1,6.958,3.332,3.240,16,9,1,-1


One-Hot encode for home_outcome

might be useful if using differnt ml model?

In [87]:
# One-hot encoding for home_outcome
##df_onehot = fb_df
##df_onehot = pd.get_dummies(fb_df, columns=['home_outcome'])

In [88]:
df_label.sample(10)

,date,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
485,06/11/2016,0,23,1,7.677,3.326,3.921,2,3,1,1
1031,24/02/2018,3,22,2,6.511,3.013,3.822,20,15,0,1
1029,12/02/2018,6,25,2,5.806,4.504,7.985,1,10,-1,-1
359,07/05/2016,12,8,2,7.315,3.491,3.891,14,11,1,1
167,20/12/2015,22,26,1,7.315,3.189,3.535,8,12,-1,1
1352,12/01/2019,5,10,1,6.522,6.755,3.386,20,16,-1,-1
790,09/09/2017,0,2,2,5.532,4.898,7.679,5,9,-1,-1
207,13/01/2016,20,18,2,7.350,3.177,3.829,9,20,0,1
335,20/04/2016,13,8,2,6.757,3.623,4.599,6,11,1,1
530,13/12/2016,2,12,2,7.657,3.301,3.020,16,1,1,0


Checking for outliers

In [89]:
df_label.describe()

,home_team,away_team,home_outcome,avg_odd_home_win,avg_odd_draw,avg_odd_away_win,home_ranking,away_ranking,last_home_result,last_away_result
count,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000,1520.000000
mean,13.687500,13.687500,1.155921,7.576486,4.878582,4.741020,10.650000,10.650000,-0.067763,0.067763
std,8.070483,8.070483,0.859462,1.715519,2.011393,3.807761,5.987578,5.987578,0.852530,0.857918
min,0.000000,0.000000,0.000000,5.324000,2.935000,2.005000,1.000000,1.000000,-1.000000,-1.000000
25%,7.000000,7.000000,0.000000,6.507000,3.290750,2.491500,5.750000,5.750000,-1.000000,-1.000000
50%,14.000000,14.000000,1.000000,7.117500,3.921500,3.285500,10.500000,10.500000,0.000000,0.000000
75%,21.000000,21.000000,2.000000,8.053250,6.617250,5.095000,15.250000,15.250000,1.000000,1.000000
max,27.000000,27.000000,2.000000,18.767000,14.597000,33.015000,20.000000,20.000000,1.000000,1.000000


In [90]:
df_label.shape

(1520, 11)

removing outliers for "avg_odd_home_win", "avg_odd_draw" and "avg_odd_away_win"

In [91]:
def remove_outliers_iqr(df, column):
    initial_count = len(df)
    
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_filtered = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    
    removed_count = initial_count - len(df_filtered)
    print(f"{removed_count} rows removed from '{column}' due to outliers.")
    
    return df_filtered

In [92]:
for col in ['avg_odd_home_win', 'avg_odd_draw', 'avg_odd_away_win']:
    df_label = remove_outliers_iqr(df_label, col)

101 rows removed from 'avg_odd_home_win' due to outliers.
6 rows removed from 'avg_odd_draw' due to outliers.
167 rows removed from 'avg_odd_away_win' due to outliers.


In [93]:
df_label.to_csv('data/cleaned-premier-league-data.csv', index=False)
